In [120]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
import sys
sys.path.append('C:/Users/bowen/Documents/Data Science from Scratch/scripts/')

In [122]:
import os
import numpy as np
import pandas as pd
from preprocessing import Preprocessor
from methods import Method, Validator

In [123]:
trainData = pd.read_csv('C:/Users/bowen/Documents/Data Science from Scratch/data/spaceship-titanic/train.csv')
testData = pd.read_csv('C:/Users/bowen/Documents/Data Science from Scratch/data/spaceship-titanic/test.csv')
trainXdata = trainData[[col for col in trainData.columns if col != 'Transported']]
testXdata = testData[[col for col in testData.columns if col != 'Transported']]
trainYData = trainData['Transported'].astype(float)

In [124]:
trainXdata[['Deck','RoomNo','ShipSide']] = trainXdata['Cabin'].str.split('/', expand=True)
trainXdata[['GroupNo', 'GroupSize']] = trainXdata['PassengerId'].str.split('_', expand=True)
trainXdata['Child'], trainXdata['Infant'] = (trainXdata['Age'] <= 18).astype(int), (trainXdata['Age'] <= 5).astype(int)
trainXdata['TotalSpent'] = np.sum(trainXdata[['RoomService', 'FoodCourt', 'ShoppingMall','Spa','VRDeck']], axis=1)

In [125]:
testXdata[['Deck','RoomNo','ShipSide']] = testXdata['Cabin'].str.split('/', expand=True)
testXdata[['GroupNo', 'GroupSize']] = testXdata['PassengerId'].str.split('_', expand=True)
testXdata['Child'], testXdata['Infant'] = (testXdata['Age'] <= 18).astype(int), (testXdata['Age'] <= 5).astype(int)
testXdata['TotalSpent'] = np.sum(testXdata[['RoomService', 'FoodCourt', 'ShoppingMall','Spa','VRDeck']], axis=1)

In [126]:
data = Preprocessor(dropCols = ['Name', 'Cabin', 'PassengerId', 'RoomNo', 'GroupNo', 'FoodCourt', 'ShoppingMall'],
                onehotCols = ['HomePlanet', 'Destination', 'ShipSide'], directCols=['VIP', 'CryoSleep', 'GroupSize'],
                ordinalCols = {'Deck': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']},
                splitterMethod='kfold', splitGroups=5, imputerMethod='lazy', normalizerMethod='zscore', bagging=True)

data(trainXdata, trainYData)

normXtrain = data.transform(trainXdata, means=data.normalizer.means, stds=data.normalizer.stds)
normXtest = data.transform(testXdata, means=data.normalizer.means, stds=data.normalizer.stds)

In [240]:
method = Method(modelMethod='logreg', lossMethod='binary_cross_entropy', optimizerMethod='gd', 
                modelParams={}, 
                lossParams={}, 
                optimizerParams={'gradThreshold': 1e-5})
validator = Validator(method=method, diagnosticParams={'metrics': ('accuracy', 'precision', 'recall')})

validator(trainXs=data.splitter.trainX, trainYs=data.splitter.trainY, 
          validationXs=data.splitter.validationX, validationYs=data.splitter.validationY, 
          binary=True, threshold=0.5)

Max epochs reached (10000)
Runtime: 19.11, Accuracy: 80.61, Precision: 80.503, Recall: 83.569
Max epochs reached (10000)
Runtime: 20.64, Accuracy: 80.552, Precision: 77.253, Recall: 84.293
Max epochs reached (10000)
Runtime: 20.95, Accuracy: 80.667, Precision: 78.832, Recall: 85.039
Max epochs reached (10000)
Runtime: 24.11, Accuracy: 78.251, Precision: 77.612, Recall: 81.25
Max epochs reached (10000)
Runtime: 19.86, Accuracy: 79.919, Precision: 77.35, Recall: 84.088
Average Runtime: 20.934, Runtime Variance: 2.93082, Average Accuracy: 80.0, Accuracy Variance: 0.83745, Average Precision: 78.31, Precision Variance: 1.52156, Average Recall: 83.648, Recall Variance: 1.66023


In [241]:
method(X=normXtrain, y=trainYData)
yHat = method.predict(normXtest)
yHat = (yHat > 0.5)*1#np.quantile(yHat, q=1-trainYData.mean())) * 1
testSubmission = pd.DataFrame(np.vstack((testData['PassengerId'], yHat)).T, columns=['PassengerId','Transported'])
testSubmission.to_csv('gd_submission.csv', index=False)

Max epochs reached (10000)


In [242]:
method = Method(modelMethod='logreg', lossMethod='binary_cross_entropy', optimizerMethod='mgd', 
                modelParams={}, 
                lossParams={}, 
                optimizerParams={'lossThreshold': 2e-6, 'maxEpochs':10000, 'gradThreshold':0})
validator = Validator(method=method, diagnosticParams={'metrics': ('accuracy', 'precision', 'recall')})

validator(trainXs=data.splitter.trainX, trainYs=data.splitter.trainY, 
          validationXs=data.splitter.validationX, validationYs=data.splitter.validationY, 
          binary=True, threshold=0.5)

Descent halted after 172 epochs as loss threshold reached (0.0 <= 2e-06)
Runtime: 2.44, Accuracy: 80.898, Precision: 81.124, Recall: 83.243
Descent halted after 323 epochs as loss threshold reached (0.0 <= 2e-06)
Runtime: 4.87, Accuracy: 80.955, Precision: 77.486, Recall: 85.012
Descent halted after 695 epochs as loss threshold reached (0.0 <= 2e-06)
Runtime: 10.71, Accuracy: 80.898, Precision: 78.564, Recall: 86.164
Descent halted after 161 epochs as loss threshold reached (0.0 <= 2e-06)
Runtime: 2.42, Accuracy: 78.481, Precision: 77.885, Recall: 81.362
Descent halted after 169 epochs as loss threshold reached (0.0 <= 2e-06)
Runtime: 2.58, Accuracy: 79.977, Precision: 77.316, Recall: 84.321
Average Runtime: 4.604, Runtime Variance: 10.18066, Average Accuracy: 80.242, Accuracy Variance: 0.90808, Average Precision: 78.475, Precision Variance: 1.93892, Average Recall: 84.02, Recall Variance: 2.66802


In [243]:
method(X=normXtrain, y=trainYData)
yHat = method.predict(normXtest)
yHat = (yHat > 0.5)*1 # np.quantile(yHat, q=1-trainYData.mean())) * 1
testSubmission = pd.DataFrame(np.vstack((testData['PassengerId'], yHat)).T, columns=['PassengerId','Transported'])
testSubmission.to_csv('mgd_submission.csv', index=False)

Descent halted after 153 epochs as loss threshold reached (0.0 <= 2e-06)


In [244]:
method = Method(modelMethod='logreg', lossMethod='binary_cross_entropy', optimizerMethod='sa', 
                modelParams={}, 
                lossParams={}, 
                optimizerParams={'lossThreshold': 1e-5, 'maxEpochs':10000, 'gradThreshold':0, 'coolingRate':1e-6,
                                 'temperature':0.05, 'initialGuess':'zeros', 'stepSize':1})

validator = Validator(method=method, diagnosticParams={'metrics': ('accuracy', 'precision', 'recall')})

validator(trainXs=data.splitter.trainX, trainYs=data.splitter.trainY, 
          validationXs=data.splitter.validationX, validationYs=data.splitter.validationY, 
          binary=True, threshold=0.5)

Max epochs reached (10000)
Runtime: 19.72, Accuracy: 78.884, Precision: 80.197, Recall: 79.761
Max epochs reached (10000)
Runtime: 18.86, Accuracy: 79.171, Precision: 76.339, Recall: 82.014
Max epochs reached (10000)
Runtime: 19.7, Accuracy: 80.725, Precision: 79.036, Recall: 84.814
Max epochs reached (10000)
Runtime: 20.43, Accuracy: 75.777, Precision: 76.477, Recall: 76.562
Max epochs reached (10000)
Runtime: 19.84, Accuracy: 78.423, Precision: 78.125, Recall: 78.397
Average Runtime: 19.71, Runtime Variance: 0.2516, Average Accuracy: 78.596, Accuracy Variance: 2.58458, Average Precision: 78.035, Precision Variance: 2.19762, Average Recall: 80.31, Recall Variance: 8.23962


In [245]:
method(X=normXtrain, y=trainYData)
yHat = method.predict(normXtest)
yHat = (yHat > 0.5)*1 #np.quantile(yHat, q=1-trainYData.mean())) * 1
testSubmission = pd.DataFrame(np.vstack((testData['PassengerId'], yHat)).T, columns=['PassengerId','Transported'])
testSubmission.to_csv('sa_submission.csv', index=False)

Max epochs reached (10000)


In [250]:
method = Method(modelMethod='logreg', lossMethod='gini', optimizerMethod='rf', 
                modelParams={}, 
                lossParams={}, 
                optimizerParams={'maxTreeDepth':10, 'minGroupSize':5, 'nCols':0.33, 'nRows':0.67, 'nTrees':50,
                                 'splitMethod':'full_sweep', 'classification': True, 'lossMethod':'mse'})

validator = Validator(method=method, diagnosticParams={'metrics': ('accuracy', 'precision', 'recall')})

validator(trainXs=data.splitter.trainX, trainYs=data.splitter.trainY, 
          validationXs=data.splitter.validationX, validationYs=data.splitter.validationY, 
          binary=True, threshold=0.5)

Runtime: 109.6, Accuracy: 80.725, Precision: 81.466, Recall: 82.263
Runtime: 135.18, Accuracy: 80.437, Precision: 78.721, Recall: 81.175
Runtime: 123.93, Accuracy: 80.092, Precision: 79.479, Recall: 82.34
Runtime: 126.65, Accuracy: 78.539, Precision: 80.301, Recall: 77.344
Runtime: 119.96, Accuracy: 80.38, Precision: 79.148, Recall: 81.998
Average Runtime: 123.064, Runtime Variance: 70.26418, Average Accuracy: 80.035, Accuracy Variance: 0.5996, Average Precision: 79.823, Precision Variance: 0.94326, Average Recall: 81.024, Recall Variance: 3.55617


In [251]:
method(X=normXtrain, y=trainYData)
yHat = method.predict(normXtest)
yHat = (yHat > 0.5) #np.quantile(yHat, q=1-trainYData.mean())) * 1
testSubmission = pd.DataFrame(np.vstack((testData['PassengerId'], yHat)).T, columns=['PassengerId','Transported'])
testSubmission.to_csv('rf_submission.csv', index=False)

In [252]:
method = Method(modelMethod='logreg', lossMethod='binary_cross_entropy', optimizerMethod='ga', 
                modelParams={}, 
                lossParams={}, 
                optimizerParams={'maxEpochs':normXtrain.shape[1]*20, 'thetaMin': -2, 'thetaMax': 2})

validator = Validator(method=method, diagnosticParams={'metrics': ('accuracy', 'precision', 'recall')})

validator(trainXs=data.splitter.trainX, trainYs=data.splitter.trainY, 
          validationXs=data.splitter.validationX, validationYs=data.splitter.validationY, 
          binary=True, threshold=0.5)

Runtime: 30.16, Accuracy: 80.265, Precision: 80.704, Recall: 82.372
Runtime: 31.44, Accuracy: 80.15, Precision: 77.627, Recall: 82.374
Runtime: 30.61, Accuracy: 80.61, Precision: 79.362, Recall: 83.915
Runtime: 30.72, Accuracy: 78.136, Precision: 78.043, Recall: 80.134
Runtime: 31.56, Accuracy: 80.207, Precision: 78.438, Recall: 82.811
Average Runtime: 30.898, Runtime Variance: 0.27826, Average Accuracy: 79.874, Accuracy Variance: 0.78046, Average Precision: 78.835, Precision Variance: 1.20301, Average Recall: 82.321, Recall Variance: 1.51386


In [253]:
method(X=normXtrain, y=trainYData)
yHat = method.predict(normXtest)
yHat = (yHat > 0.5) #np.quantile(yHat, q=1-trainYData.mean())) * 1
testSubmission = pd.DataFrame(np.vstack((testData['PassengerId'], yHat)).T, columns=['PassengerId','Transported'])
testSubmission.to_csv('ga_submission.csv', index=False)

In [117]:
# method = Method(modelMethod='logreg', lossMethod='mse', optimizerMethod='gb', 
#                 modelParams={}, 
#                 lossParams={}, 
#                 optimizerParams={'maxTreeDepth':2, 'minGroupSize':5, 'maxBoosterDepth':10,
#                                  'splitMethod':'histogram', 'classification': False, 'lossMethod':'mse', 'lr':0.1})

# validator = Validator(method=method, diagnosticParams={'metrics': ('accuracy', 'precision', 'recall')})

# validator(trainXs=data.splitter.trainX, trainYs=data.splitter.trainY, 
#           validationXs=data.splitter.validationX, validationYs=data.splitter.validationY, 
#           binary=True, threshold=0.5)

iter 0: prob mean=0.503, grad std=0.500, yHat std=0.008
iter 1: prob mean=0.503, grad std=0.500, yHat std=0.015
iter 2: prob mean=0.503, grad std=0.499, yHat std=0.024
iter 3: prob mean=0.503, grad std=0.499, yHat std=0.033
iter 4: prob mean=0.503, grad std=0.498, yHat std=0.042
iter 5: prob mean=0.503, grad std=0.498, yHat std=0.051
iter 6: prob mean=0.503, grad std=0.497, yHat std=0.059
iter 7: prob mean=0.503, grad std=0.497, yHat std=0.068
iter 8: prob mean=0.503, grad std=0.497, yHat std=0.076
iter 9: prob mean=0.503, grad std=0.496, yHat std=0.084
Runtime: 3.03, Accuracy: 55.121, Precision: 53.368, Recall: 92.412
iter 0: prob mean=0.502, grad std=0.500, yHat std=0.006
iter 1: prob mean=0.502, grad std=0.500, yHat std=0.013
iter 2: prob mean=0.502, grad std=0.500, yHat std=0.019
iter 3: prob mean=0.502, grad std=0.499, yHat std=0.025
iter 4: prob mean=0.502, grad std=0.499, yHat std=0.030
iter 5: prob mean=0.502, grad std=0.499, yHat std=0.036
iter 6: prob mean=0.502, grad std=0.4

In [254]:
submissionFiles = [x for x in os.listdir() if '_submission.csv' in x and 'ensemble' not in x]
print(submissionFiles)
yPreds = np.zeros((len(submissionFiles), yHat.shape[0]))
for i in range(len(submissionFiles)):
    yPreds[i] = (pd.read_csv(submissionFiles[i])['Transported'])
    
testSubmission = pd.DataFrame(np.vstack((testData['PassengerId'], np.round(np.mean(yPreds, axis=0))>0)).T, 
                              columns=['PassengerId','Transported'])
testSubmission.to_csv('ensemble_submission.csv', index=False)

agree = np.round((1 - np.sum(np.sum(np.abs(yPreds - np.mean(yPreds, axis=0)), axis=0) > 0) / yHat.shape[0]) * 100, 2)

print(f'Percent Perfect Agreement: {agree}%')
print(f'Total Variance Between Methods: {np.round(np.mean(np.var(yPreds, axis=0)), 4)}')

['ga_submission.csv', 'gd_submission.csv', 'mgd_submission.csv', 'rf_submission.csv', 'sa_submission.csv']
Percent Perfect Agreement: 86.98%
Total Variance Between Methods: 0.0224
